In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# leyendo datos de Real Madrid
path ='https://data.insideairbnb.com/spain/comunidad-de-madrid/madrid/2025-06-12/data/listings.csv.gz'
df = pd.read_csv(path)

df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,21853,https://www.airbnb.com/rooms/21853,20250612050748,2025-06-26,city scrape,Bright and airy room,We have a quiet and sunny room with a good vie...,We live in a leafy neighbourhood with plenty o...,https://a0.muscache.com/pictures/68483181/87bc...,83531,...,4.82,4.21,4.67,NaN,f,2,0,2,0,0.25
1,30320,https://www.airbnb.com/rooms/30320,20250612050748,2025-06-27,previous scrape,Apartamentos Dana Sol,NaN,NaN,https://a0.muscache.com/pictures/336868/f67409...,130907,...,4.78,4.90,4.69,NaN,f,3,3,0,0,0.94
2,30959,https://www.airbnb.com/rooms/30959,20250612050748,2025-06-27,previous scrape,Beautiful loft in Madrid Center,Beautiful Loft 60m2 size just in the historica...,NaN,https://a0.muscache.com/pictures/78173471/835e...,132883,...,4.63,4.88,4.25,NaN,f,1,1,0,0,0.06
3,40916,https://www.airbnb.com/rooms/40916,20250612050748,2025-06-26,previous scrape,Apartasol Apartamentos Dana,NaN,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,130907,...,4.79,4.88,4.55,NaN,t,3,3,0,0,0.27
4,62423,https://www.airbnb.com/rooms/62423,20250612050748,2025-06-25,city scrape,MAGIC ARTISTIC HOUSE IN THE CENTER OF MADRID,INCREDIBLE HOME OF AN ARTIST SURROUNDED BY PAI...,DISTRICT WITH VERY GOOD VIBES IN THE MIDDLE OF...,https://a0.muscache.com/pictures/miso/Hosting-...,303845,...,4.86,4.97,4.59,NaN,f,3,1,2,0,2.70


In [35]:
#HOSPEDAJES CON MAS DE UNA HABITACION QUE ESTAN CERCA A CENTROS COMERCIALES
# Importar librerías
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster
from geopy.distance import geodesic

# Cargar datos desde Airbnb Madrid
url = "https://data.insideairbnb.com/spain/comunidad-de-madrid/madrid/2025-06-12/data/listings.csv.gz"
df = pd.read_csv(url, compression='gzip', low_memory=False)

# Filtrar hospedajes con más de 2 camas
df_filtrado = df[df['bedrooms'] > 2].copy()

# Eliminar filas sin coordenadas
df_filtrado = df_filtrado.dropna(subset=['latitude', 'longitude'])

# Crear lista de coordenadas de centros comerciales (aproximadas)
centros_comerciales = [
    {"nombre": "Centro Comercial La Vaguada", "coords": (40.4781, -3.7063)},
    {"nombre": "Centro Comercial Príncipe Pío", "coords": (40.4204, -3.7202)},
    {"nombre": "Centro Comercial Plenilunio", "coords": (40.4386, -3.6013)},
    {"nombre": "Centro Comercial Islazul", "coords": (40.3677, -3.7394)},
    {"nombre": "Centro Comercial Gran Vía de Hortaleza", "coords": (40.4719, -3.6426)},
]

# Filtrar hospedajes cercanos a centros comerciales (radio de 1 km)
def cerca_de_centro(lat, lon):
    punto = (lat, lon)
    for cc in centros_comerciales:
        if geodesic(punto, cc["coords"]).km <= 1.0:
            return cc["nombre"]
    return None

df_filtrado["centro_comercial_cercano"] = df_filtrado.apply(
    lambda row: cerca_de_centro(row["latitude"], row["longitude"]), axis=1
)

# Filtrar solo hospedajes que estén cerca de algún centro comercial
df_cercanos = df_filtrado.dropna(subset=["centro_comercial_cercano"])

# Crear el mapa centrado en Madrid
mapa = folium.Map(location=[40.4168, -3.7038], zoom_start=12)

# Añadir marcadores de centros comerciales
for cc in centros_comerciales:
    folium.Marker(
        location=cc["coords"],
        popup=f" {cc['nombre']}",
        icon=folium.Icon(color="green", icon="shopping-cart", prefix="fa")
    ).add_to(mapa)

# Añadir hospedajes con FastMarkerCluster
locations = list(zip(df_cercanos.latitude, df_cercanos.longitude))
FastMarkerCluster(data=locations).add_to(mapa)

# Mostrar el mapa
mapa

In [36]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster

#Cargar dataset desde Inside Airbnb (Madrid)
url = 'https://data.insideairbnb.com/spain/comunidad-de-madrid/madrid/2025-06-12/data/listings.csv.gz'
df = pd.read_csv(url, compression='gzip')

#Filtrar hospedajes con más de 2 camas
df['beds'] = pd.to_numeric(df['beds'], errors='coerce')
df_filtrado = df[df['beds'] > 2].copy()

print(f"Hospedajes con más de 2 camas: {len(df_filtrado)}")

#Crear el mapa base centrado en Madrid
map2 = folium.Map(location=[40.4168, -3.7038], zoom_start=12, tiles='CartoDB positron')

#Crear la lista de ubicaciones con texto emergente
#    (latitude, longitude, popup_html)
locations = [
    [
        row['latitude'],
        row['longitude'],
        f"<b>{row.get('name', 'Sin nombre')}</b><br>"
        f"Tipo: {row.get('property_type', 'Desconocido')}<br>"
        f"Camas: {row.get('beds', 'N/A')}<br>"
        f"<a href='{row.get('listing_url', '#')}' target='_blank'>Ver en Airbnb</a>"
    ]
    for _, row in df_filtrado.iterrows()
]

#Agregar los puntos al mapa con clúster rápido
FastMarkerCluster(
    data=[(lat, lon, popup) for lat, lon, popup in locations]
).add_to(map2)

#Mostrar el mapa (en Jupyter o Colab se renderiza directamente)
map2

Hospedajes con más de 2 camas: 4639


In [37]:
df_filtrado.head(3)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
6,72150,https://www.airbnb.com/rooms/72150,20250612050748,2025-06-26,city scrape,Sunny attic duplex flat with terrace next to Sol,"Nestled in the serene heart of Madrid, this re...",NaN,https://a0.muscache.com/pictures/5a7618cb-bd61...,364585,...,4.85,4.84,4.48,VT-6993,f,4,4,0,0,0.76
9,86042,https://www.airbnb.com/rooms/86042,20250612050748,2025-06-27,city scrape,Cozy attic with intimate rooftop terrace+ elev...,"Nestled in Madrid's tranquil heart, this apart...",NaN,https://a0.muscache.com/pictures/12207675/648f...,364585,...,4.92,4.92,4.68,VT-3974,f,4,4,0,0,1.36
17,628394,https://www.airbnb.com/rooms/628394,20250612050748,2025-06-26,city scrape,"Spectacular Triplex in Canaleja, Sol",Triplex flat with 4 double bedrooms located a ...,This triplex is located in the heart of Madrid...,https://a0.muscache.com/pictures/hosting/Hosti...,3121587,...,4.74,4.87,4.49,VT-M-3436,t,3,3,0,0,0.56


In [38]:
df_filtrado['price']

,price
6,$196.00
9,$209.00
17,$448.00
31,$146.00
32,$134.00
...,...
25983,$156.00
25990,$295.00
25995,$95.00
25998,$131.00


In [65]:
# vamos a lo que nos interesa $ (La plata)
def limpiar(a):
  return str(a).replace("$","").replace(",","")

#df["price"] = df["price"].apply(limpiar).astype(float)
df_filtrado['price'] = df_filtrado['price'].str.replace('$', '').str.replace(',', '').astype(float)

In [66]:
df_filtrado['price']

,price
9,209.0
17,448.0
37,190.0
45,118.0
68,302.0
...,...
25983,156.0
25990,295.0
25995,95.0
25998,131.0


In [67]:
# listar los precios de 100 a 200 en df_filtrado
df_filtrado = df_filtrado[(df_filtrado['price'] >= 100) & (df_filtrado['price'] <= 200)]


In [54]:
df_filtrado['price']

,price
6,196.0
31,146.0
37,190.0
45,118.0
50,118.0
...,...
25931,110.0
25932,163.0
25983,156.0
25998,131.0


In [43]:
df_filtrado.head(3)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
6,72150,https://www.airbnb.com/rooms/72150,20250612050748,2025-06-26,city scrape,Sunny attic duplex flat with terrace next to Sol,"Nestled in the serene heart of Madrid, this re...",NaN,https://a0.muscache.com/pictures/5a7618cb-bd61...,364585,...,4.85,4.84,4.48,VT-6993,f,4,4,0,0,0.76
31,685152,https://www.airbnb.com/rooms/685152,20250612050748,2025-06-26,city scrape,house-hotel,NaN,The one where the chalet is located is a quiet...,https://a0.muscache.com/pictures/15d25840-e367...,3494012,...,4.35,4.66,4.34,NaN,f,2,2,0,0,1.71
32,688795,https://www.airbnb.com/rooms/688795,20250612050748,2025-06-25,city scrape,Hab Penthouse Plaza Mayor - Felipe III Balcony,"Temporary rental, non-tourist. Short term.",NEIGHBOURHOOD & LOCATION:<br /><br />Here in t...,https://a0.muscache.com/pictures/miso/Hosting-...,1732442,...,4.88,4.94,4.60,ESHFNT0000280910001026610030000000000000000000...,f,6,1,5,0,1.79


In [44]:
df_filtrado.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [51]:
# visualizar los que tienen mayor a 2 camas en df_filtrado en el campo bedrooms
df_filtrado = df_filtrado[df_filtrado['beds'] > 2]


In [59]:
# visualizar los que tienen mayor a 2 y menor a 5
df_filtrado = df_filtrado[(df_filtrado['beds'] > 2) & (df_filtrado['beds'] < 5)]

In [61]:
df_filtrado = df_filtrado[df_filtrado['room_type'] == 'Entire home/apt']

In [50]:
df_filtrado['room_type']

,room_type
6,Entire home/apt
31,Entire home/apt
37,Entire home/apt
45,Entire home/apt
50,Entire home/apt
...,...
25931,Entire home/apt
25932,Entire home/apt
25983,Entire home/apt
25998,Entire home/apt


In [53]:
df_filtrado.head(3)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
6,72150,https://www.airbnb.com/rooms/72150,20250612050748,2025-06-26,city scrape,Sunny attic duplex flat with terrace next to Sol,"Nestled in the serene heart of Madrid, this re...",NaN,https://a0.muscache.com/pictures/5a7618cb-bd61...,364585,...,4.85,4.84,4.48,VT-6993,f,4,4,0,0,0.76
31,685152,https://www.airbnb.com/rooms/685152,20250612050748,2025-06-26,city scrape,house-hotel,NaN,The one where the chalet is located is a quiet...,https://a0.muscache.com/pictures/15d25840-e367...,3494012,...,4.35,4.66,4.34,NaN,f,2,2,0,0,1.71
37,198296,https://www.airbnb.com/rooms/198296,20250612050748,2025-06-27,city scrape,Very quiet central top floor with 2 terraces,Beautiful central top floor apartment with two...,Malasaña is a Castilian neighborhood in the ce...,https://a0.muscache.com/pictures/miso/Hosting-...,967721,...,4.91,4.98,4.68,NaN,f,2,1,1,0,0.54


In [48]:
# listar los distintos
df_filtrado['room_type'].unique()

array(['Entire home/apt', 'Private room', 'Shared room'], dtype=object)

In [69]:
# cantidad de registro de df_filtro
df_filtrado.shape

(1970, 79)

In [70]:
df_filtrado.head(3)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
37,198296,https://www.airbnb.com/rooms/198296,20250612050748,2025-06-27,city scrape,Very quiet central top floor with 2 terraces,Beautiful central top floor apartment with two...,Malasaña is a Castilian neighborhood in the ce...,https://a0.muscache.com/pictures/miso/Hosting-...,967721,...,4.91,4.98,4.68,NaN,f,2,1,1,0,0.54
45,716292,https://www.airbnb.com/rooms/716292,20250612050748,2025-06-26,city scrape,Just in the centre: Malasaña-Chueca,My house is your house! Just in the center of ...,NaN,https://a0.muscache.com/pictures/29777389/dda3...,3691303,...,4.84,4.89,4.77,ESFCNT0000281080003461070000000000000000000000...,f,1,1,0,0,0.84
70,263748,https://www.airbnb.com/rooms/263748,20250612050748,2025-06-26,city scrape,"LOFT ""C"" cerca de IFEMA Madrid",The loft is an open space with different heigh...,LOFT stay in a unique space a nice experience....,https://a0.muscache.com/pictures/2604888/ae7f9...,1351865,...,4.89,4.32,4.64,NaN,t,2,2,0,0,1.04


In [71]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster

#Crear el mapa base centrado en Madrid
map2 = folium.Map(location=[40.4168, -3.7038], zoom_start=12, tiles='CartoDB positron')

#Crear la lista de ubicaciones con texto emergente
#    (latitude, longitude, popup_html)
locations = [
    [
        row['latitude'],
        row['longitude'],
        f"<b>{row.get('name', 'Sin nombre')}</b><br>"
        f"Tipo: {row.get('property_type', 'Desconocido')}<br>"
        f"Camas: {row.get('beds', 'N/A')}<br>"
        f"<a href='{row.get('listing_url', '#')}' target='_blank'>Ver en Airbnb</a>"
    ]
    for _, row in df_filtrado.iterrows()
]

#Agregar los puntos al mapa con clúster rápido
FastMarkerCluster(
    data=[(lat, lon, popup) for lat, lon, popup in locations]
).add_to(map2)

#Mostrar el mapa (en Jupyter o Colab se renderiza directamente)
map2

In [72]:
df_filtrado.head(3)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
37,198296,https://www.airbnb.com/rooms/198296,20250612050748,2025-06-27,city scrape,Very quiet central top floor with 2 terraces,Beautiful central top floor apartment with two...,Malasaña is a Castilian neighborhood in the ce...,https://a0.muscache.com/pictures/miso/Hosting-...,967721,...,4.91,4.98,4.68,NaN,f,2,1,1,0,0.54
45,716292,https://www.airbnb.com/rooms/716292,20250612050748,2025-06-26,city scrape,Just in the centre: Malasaña-Chueca,My house is your house! Just in the center of ...,NaN,https://a0.muscache.com/pictures/29777389/dda3...,3691303,...,4.84,4.89,4.77,ESFCNT0000281080003461070000000000000000000000...,f,1,1,0,0,0.84
70,263748,https://www.airbnb.com/rooms/263748,20250612050748,2025-06-26,city scrape,"LOFT ""C"" cerca de IFEMA Madrid",The loft is an open space with different heigh...,LOFT stay in a unique space a nice experience....,https://a0.muscache.com/pictures/2604888/ae7f9...,1351865,...,4.89,4.32,4.64,NaN,t,2,2,0,0,1.04


que los 2 mapas esten al final